In [22]:
from sklearn.decomposition import PCA
from tdamapper.core import MapperAlgorithm
from tdamapper.cover import CubicalCover
from tdamapper.plot import MapperLayoutInteractive
from sklearn.cluster import DBSCAN, AgglomerativeClustering
from tdamapper.clustering import FailSafeClustering
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import cdist

import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn import metrics
import pandas as pd
import numpy as np
from sklearn.metrics import pairwise_distances_argmin_min, silhouette_score

from functions import *
from chi import *
from regressionP import *

In [6]:
def preprocess(input_data, sample = 592):
    sample_data = input_data[input_data['當事者順位'] == 1].reset_index(drop=True, inplace=False).sample(sample).reset_index(drop=True)
    dataA = sample_data[select_lst]
    
    death_injury_data = split_death_injury(dataA['死亡受傷人數'])
    dist_df = pd.concat([dataA, death_injury_data], axis=1)
    dist_df.drop(columns=['死亡受傷人數'], inplace=True)
    
    return dist_df, sample_data

def get_calinski_from_db(input_data, eps): 
    X = input_data.iloc[:, 3:6]

    db = DBSCAN(eps=eps, min_samples=10).fit(X)
    labels = db.labels_
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

    input_data['label'] = labels
    
    unique_labels = np.unique(labels)
    colors = plt.cm.Spectral(np.linspace(0, 1, len(unique_labels)))

    if len(set(labels)) != 1:
        score = metrics.calinski_harabasz_score(X, labels)
        silhouette_score_value = silhouette_score(X, labels)
    else:
        score = -1
        silhouette_score_value = -1
        
    return score, input_data, db, labels, n_clusters_, silhouette_score_value, unique_labels, colors

def latlon_to_xyz(lat, lon):
    lat_rad = np.radians(lat)
    lon_rad = np.radians(lon)
    x = np.cos(lat_rad) * np.cos(lon_rad)
    y = np.cos(lat_rad) * np.sin(lon_rad)
    z = np.sin(lat_rad)
    return np.vstack((x, y, z)).T

# 計算球面距離
def spherical_dist(pos1, pos2, radius=6371):
    cos_angle = np.dot(pos1, pos2.T)
    angle = np.arccos(np.clip(cos_angle, -1, 1))
    return radius * angle

def calculate_distances(df, scenic_df, colname, distance_threshold=500):
    # 將經緯度轉換成球體的三維坐標
    df_xyz = latlon_to_xyz(df['緯度'], df['經度'])
    scenic_xyz = latlon_to_xyz(scenic_df['緯度'], scenic_df['經度'])

    # 計算所有配對之間的球面距離，並轉換為米
    distances = spherical_dist(df_xyz, scenic_xyz) * 1000

    # 檢查哪些距離小於設定的threshold
    distances_less_than_threshold = (distances < distance_threshold)

    # 計算每一行小於閾值的點數量
    df[colname] = distances_less_than_threshold.sum(axis=1)

    return df

In [7]:
data1 = pd.read_csv("./Data/NPA_TMA2_1.csv", low_memory=False)[:-2]
data2 = pd.read_csv("./Data/NPA_TMA2_2.csv", low_memory=False)[:-2]
data3 = pd.read_csv("./Data/NPA_TMA2_3.csv", low_memory=False)[:-2]
data4 = pd.read_csv("./Data/NPA_TMA2_4.csv", low_memory=False)[:-2]
dataA2 = pd.concat([data1, data2, data3, data4], ignore_index=True)
dataA1 = pd.read_csv("./Data/NPA_TMA1.csv")[:-2]
# Qmap
school_data = pd.read_csv('CalculatedData/coord/school_data.csv')
train_data = pd.read_csv('CalculatedData/coord/train_data.csv')
post_data = pd.read_csv('CalculatedData/coord/post_data.csv')
museum_data = pd.read_csv('CalculatedData/coord/museum_data.csv')
guesthouse_data = pd.read_csv('CalculatedData/coord/guesthouse_data.csv')
themepark_data = pd.read_csv('CalculatedData/coord/themepark_data.csv')
supermarket_data = pd.read_csv('CalculatedData/coord/supermarket_data.csv')
coffee_data = pd.read_csv('CalculatedData/coord/coffee_data.csv')
fastfood_data = pd.read_csv('CalculatedData/coord/fastfood_data.csv')
rtc_data = pd.read_csv('CalculatedData/coord/rtc_data.csv')
thsrc_data = pd.read_csv('CalculatedData/coord/thsrc_data.csv')
baseball_data = pd.read_csv('CalculatedData/coord/baseball_data.csv')
night_data = pd.read_csv('CalculatedData/coord/night_data.csv')
port_data = pd.read_csv('CalculatedData/coord/port_data.csv')
library_data = pd.read_csv('CalculatedData/coord/library_data.csv')
bank_data = pd.read_csv('CalculatedData/coord/bank_data.csv')
hospital_data = pd.read_csv('CalculatedData/coord/hospital_data.csv')
temple_data = pd.read_csv('CalculatedData/coord/temple_data.csv')
police_data = pd.read_csv('CalculatedData/coord/police_data.csv')
gas_data = pd.read_csv('CalculatedData/coord/gas_data.csv')
town_office_data = pd.read_csv('CalculatedData/coord/town_office_data.csv')
hr_office_data = pd.read_csv('CalculatedData/coord/hr_office_data.csv')
mv_data = pd.read_csv('CalculatedData/coord/mv_data.csv')
ntb_data = pd.read_csv('CalculatedData/coord/ntb_data.csv')
# 開放平台
scenic = pd.read_csv("./Data/Scenic_Spot_C_f.csv")

In [8]:
# List of columns to select
select_lst = [
    '天候名稱', 
    '路面狀況-路面狀態名稱',
    '肇因研判大類別名稱-主要', '當事者屬-性-別名稱', '當事者事故發生時年齡',
    '車輛撞擊部位大類別名稱-最初',
    '光線名稱',
    '道路類別-第1當事者-名稱',
    '速限-第1當事者',
    '道路型態大類別名稱',
    '事故位置大類別名稱', 
    '號誌-號誌種類名稱',
    '車道劃分設施-分向設施大類別名稱', '車道劃分設施-分道設施-快車道或一般車道間名稱',
    '車道劃分設施-分道設施-快慢車道間名稱', '車道劃分設施-分道設施-路面邊線名稱',
    '事故類型及型態大類別名稱',
    '死亡受傷人數',
    '經度', '緯度',
]

In [9]:
dist_dfA1 = preprocess(dataA1, sample = 592)
dist_dfA2 = preprocess(dataA2, sample = 5920)

data_frames = {
    '學校': school_data,
    # '火車站': train_data,
    # '郵局': post_data,
    # '博物館': museum_data,
    # '民宿': guesthouse_data,
    # '遊樂園': themepark_data,
    # '大賣場': supermarket_data,
    # '咖啡店': coffee_data,
    # '速食店': fastfood_data,
    # '捷運車站': rtc_data,
    # '高鐵車站': thsrc_data,
    # '棒球場': baseball_data,
    # '夜市': night_data,
    # '漁港': port_data,
    # '圖書館': library_data,
    # '銀行': bank_data,
    # '醫院': hospital_data,
    # '寺廟': temple_data,
    # '警察局': police_data,
    # '加油站': gas_data,
    # '公所': town_office_data,
    # '戶政事務所': hr_office_data,
    # '監理站': mv_data,
    # '國稅局': ntb_data,
}
for colname, data_frame in data_frames.items():
    scenic_dfA1 = calculate_distances(dist_dfA1[0], data_frame, colname=colname)
    scenic_dfA2 = calculate_distances(dist_dfA2[0], data_frame, colname=colname)
    
rbind_data = pd.concat([scenic_dfA1, scenic_dfA2], axis=0, ignore_index=True)

for col in data_frames.keys():
    rbind_data.loc[rbind_data[col] > 1, col] = 2
rbind_data.loc[rbind_data['受傷'] > 1, '受傷'] = 2
rbind_data['速限-第1當事者'] = rbind_data['速限-第1當事者'].apply(lambda x: 1 if x > 60 else 0)
rbind_data = process_age(rbind_data)

dist_df = process_data(rbind_data)
scaler = StandardScaler()

full_dist = pd.DataFrame(scaler.fit_transform(dist_df), columns = dist_df.columns)
X1 = full_dist.drop(['受傷', '死亡', '經度', '緯度'], axis=1).to_numpy()

In [6]:
def find_ratio(input_data, components) :
    best_comp = {}
    for comp in range(1,components+1):   
        pca = PCA(comp).fit(input_data)
        
        best_comp[comp] = pca.explained_variance_ratio_.sum()
        
    max_comp = max(best_comp, key=best_comp.get)  # 使用 key=best_comp.get 找到最大值的鍵
    print("最佳成分數：", max_comp)
    print("解釋方差比率累計值：", best_comp[max_comp])

# lens1 = find_ratio(X1, 30)

In [10]:
lens1 = PCA(10).fit_transform(X1)

mapper_algo1 = MapperAlgorithm(
    cover = CubicalCover(
        n_intervals = 3,
        overlap_frac = 0.3
    ),
    clustering = FailSafeClustering(
        clustering = AgglomerativeClustering(3, affinity='euclidean', linkage='ward'),
        verbose = False)
)

mapper_graph1 = mapper_algo1.fit_transform(X1, lens1)

In [25]:
# mapper_plot1 = MapperLayoutInteractive(
#     mapper_graph1,
#     colors = dist_df[['路面狀況-路面狀態名稱']].to_numpy(),
#     cmap = 'jet',
#     # agg = np.nanmean,
#     agg = most_frequent_nonan,
#     dim = 3,
#     iterations = 30,
#     seed = 5,
#     width = 800,
#     height = 500)

# fig_mean1 = mapper_plot1.plot()
# fig_mean1.show(config={'scrollZoom': True})

In [14]:
x = vars(mapper_plot1._MapperLayoutInteractive__fig)['_data_objs'][1]['x']
y = vars(mapper_plot1._MapperLayoutInteractive__fig)['_data_objs'][1]['y']
z = vars(mapper_plot1._MapperLayoutInteractive__fig)['_data_objs'][1]['z']

threeDimData = pd.DataFrame({'x': x, 'y': y, 'z': z})

import re
data_tuple = vars(mapper_plot1._MapperLayoutInteractive__fig)['_data_objs'][1]['text']

data = []
for item in data_tuple:
    color = int(re.search(r'color: (\d+)', item).group(1))
    node = int(re.search(r'node: (\d+)', item).group(1))
    size = int(re.search(r'size: (\d+)', item).group(1))
    data.append({'color': color, 'node': node, 'size': size})
component_info = pd.DataFrame(data)

full_info = pd.concat([component_info, threeDimData], axis=1)

mp_content_origin = vars(mapper_plot1._MapperLayoutInteractive__graph)['_node']

mp_content = pd.DataFrame.from_dict(mp_content_origin, orient='index')
mp_content.reset_index(inplace=True)
mp_content.rename(columns={'index': 'node'}, inplace=True)

full_info = pd.merge(full_info, mp_content, on=['node', 'size'], how='inner')

In [26]:
# calinski_data = get_calinski_from_db(full_info, 0.025)
# labels = calinski_data[3]
# db = calinski_data[2]
# n_clusters_ = calinski_data[4]

# unique_labels = set(labels)
# core_samples_mask = np.zeros_like(labels, dtype=bool)
# core_samples_mask[db.core_sample_indices_] = True

# def matplotlib_to_plotly(cmap, alpha=1):
#     """rgba"""
#     return f'rgba({int(cmap[0]*200)}, {int(cmap[1]*200)}, {int(cmap[2]*200)}, {alpha})'

# # colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]  
# colors = [matplotlib_to_plotly(plt.cm.Spectral(each), alpha=0.8) for each in np.linspace(0, 1, len(unique_labels))]
# fig = go.Figure()

# for k, col in zip(unique_labels, colors):
#     if k == -1:
#         # col = 'rgba(0,0,0,0)'
#         col = 'rgba(0,0,0,0)'

#     class_member_mask = labels == k

#     core_samples = full_info.iloc[:, 3:6][class_member_mask & core_samples_mask]
#     fig.add_trace(go.Scatter3d(
#         x=core_samples.iloc[:, 0],
#         y=core_samples.iloc[:, 1],
#         z=core_samples.iloc[:, 2],
#         mode='markers',
#         marker=dict(
#             size=6,
#             color=col,
#             opacity=0.8
#         ),
#         name=f'Cluster {k} Core'
#     ))

#     non_core_samples = full_info.iloc[:, 3:6][class_member_mask & ~core_samples_mask]
#     fig.add_trace(go.Scatter3d(
#         x=non_core_samples.iloc[:, 0],
#         y=non_core_samples.iloc[:, 1],
#         z=non_core_samples.iloc[:, 2],
#         mode='markers',
#         marker=dict(
#             size=6,
#             color=col,
#             opacity=0.5
#         ),
#         name=f'Cluster {k} Non-Core'
#     ))

# fig.update_layout(
#     title=f"Estimated number of clusters: {n_clusters_}",
#     margin=dict(l=0, r=0, b=0, t=0)
# )

# fig.show()

In [17]:
from chi import *

label_0 = full_info[full_info['label'] == 0]
label_1 = full_info[full_info['label'] == 1]
label_2 = full_info[full_info['label'] == 2]

count_0 = get_count_dict(label_0)
count_1 = get_count_dict(label_1)
count_2 = get_count_dict(label_2)

print(full_info['label'].unique())

[ 1  0 -1  2]


In [18]:
full_0 = rbind_data.loc[count_0.keys()]
full_1 = rbind_data.loc[count_1.keys()]
full_2 = rbind_data.loc[count_2.keys()]

lst01 = list(count_0.keys() & count_1.keys())
lst02 = list(count_0.keys() & count_2.keys())
lst12 = list(count_1.keys() & count_2.keys())
# 將重複的key另外拉出進行分析，這裡drop是為了符合卡方的獨立性前提假設
full_01 = full_0.loc[lst01]
full_02 = full_0.loc[lst02]
full_12 = full_1.loc[lst12]

full_0 = full_0.drop(lst01, errors='ignore')
full_0 = full_0.drop(lst02, errors='ignore')
full_1 = full_1.drop(lst01, errors='ignore')
full_1 = full_1.drop(lst12, errors='ignore')
full_2 = full_2.drop(lst02, errors='ignore')
full_2 = full_2.drop(lst12, errors='ignore')

print('01連接點數量', len(lst01))
for key1 in lst01:
    del count_0[key1]
    del count_1[key1]
print('02連接點數量', len(lst02))
for key2 in lst02:
    del count_0[key2]
    del count_2[key2]
print('12連接點數量', len(lst12))
for key3 in lst12:
    del count_1[key3]
    del count_2[key3]

full_0 = add_count(full_0, count_0)
full_1 = add_count(full_1, count_1)
full_2 = add_count(full_2, count_2)

print('各分群相加', full_0.shape[0] + full_1.shape[0] + full_2.shape[0])
print('各分群大小', full_0.shape, full_1.shape, full_2.shape)
# print('權重', full_0['count'].sum(), full_1['count'].sum(), full_2['count'].sum())

01連接點數量 17
02連接點數量 18
12連接點數量 0
各分群相加 6214
各分群大小 (5628, 23) (540, 23) (46, 23)


In [20]:
lst_regression = [
    '天候名稱',
    '路面狀況-路面狀態名稱',
    '肇因研判大類別名稱-主要', '當事者屬-性-別名稱', '當事者事故發生時年齡', 
    '車輛撞擊部位大類別名稱-最初',
    '光線名稱',
    '道路類別-第1當事者-名稱', 
    '速限-第1當事者', 
    '道路型態大類別名稱', 
    '事故位置大類別名稱',
    '號誌-號誌種類名稱',
    '車道劃分設施-分向設施大類別名稱', '車道劃分設施-分道設施-快車道或一般車道間名稱',
    '車道劃分設施-分道設施-快慢車道間名稱', '車道劃分設施-分道設施-路面邊線名稱',
    '事故類型及型態大類別名稱',
]

def calculate_proportions(full, category_column):
    # 計算受傷比例
    grouped1 = full.groupby([category_column, '受傷']).size().unstack(fill_value=0)
    total_count1 = grouped1.sum(axis=1)
    proportions1 = grouped1.div(total_count1, axis=0) * 100
    proportions1 = proportions1.round(2)  # 四捨五入到小數點後兩位
    proportions1.columns = [f'受傷{i}' for i in range(grouped1.shape[1])]  # 更新列名稱

    # 計算死亡比例
    grouped2 = full.groupby([category_column, '死亡']).size().unstack(fill_value=0)
    total_count2 = grouped2.sum(axis=1)
    proportions2 = grouped2.div(total_count2, axis=0) * 100
    proportions2 = proportions2.round(2)  # 四捨五入到小數點後兩位
    proportions2.columns = [f'死亡{i}' for i in range(grouped2.shape[1])]  # 更新列名稱

    # 合併兩個 DataFrame
    final_df = proportions1.join(proportions2)
    final_df['總數'] = total_count1
    # 重置索引以將 category_column 作為一個普通列
    final_df.reset_index(inplace=True)

    return final_df

In [23]:
X01, y01, p01 = pval(full_0, full_1, lst_regression)
p01[p01['p_value'] < 0.05]

/Users/wangqiqian/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge



,coefficients,standard_error,wald_statistics,p_value,feature
路面狀況-路面狀態名稱,1.983207,0.253039,7.837561,4.662937e-15,路面狀況-路面狀態名稱
天候名稱,1.266694,0.411686,3.076845,2.092037e-03,天候名稱


In [27]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calculate_vif(X):
    vif = pd.DataFrame()
    vif["features"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif

# 假设 c0_for_lm_X 是你的预测变量DataFrame
vif_df = calculate_vif(X01[lst_regression])
print(vif_df)

                   features       VIF
0                      天候名稱  2.852214
1               路面狀況-路面狀態名稱  2.844998
2              肇因研判大類別名稱-主要  1.078164
3                當事者屬-性-別名稱  1.022823
4                當事者事故發生時年齡  1.021078
5            車輛撞擊部位大類別名稱-最初  1.060627
6                      光線名稱  1.023345
7             道路類別-第1當事者-名稱  1.038176
8                  速限-第1當事者       NaN
9                 道路型態大類別名稱  7.722232
10                事故位置大類別名稱  7.641292
11                號誌-號誌種類名稱  1.213776
12         車道劃分設施-分向設施大類別名稱  1.079623
13  車道劃分設施-分道設施-快車道或一般車道間名稱  1.208093
14      車道劃分設施-分道設施-快慢車道間名稱  1.095880
15       車道劃分設施-分道設施-路面邊線名稱  1.164547
16             事故類型及型態大類別名稱  1.101906


/Users/wangqiqian/opt/anaconda3/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1738: RuntimeWarning:

invalid value encountered in scalar divide

